In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")

#### Original Dataset

In [ ]:
df = pd.read_excel('original_dataset.xlsx')
df.info()

#### Data Summarization

In [ ]:
# set the summary dataframe
summary = pd.DataFrame(columns=['latest_edit_year_-_2018', 'num_edits_2017', 'edits_change_2017_2018'], 
                       index=['coverage', 'min', 'P1', 'P5', 'P10', 'Q1', 'median', 
                              'Q3', 'P90', 'P95', 'P99', 'max', 'mean', 'q-range', 
                              'skew', 'kurt', 'zero', 'positive', 'negative', 'missing'])
summary = summary.astype('float')
# define the summary function
def get_summary(summary, df):
    # coverage
    for i in summary.columns:
        summary[i]['coverage'] = df.describe().loc['count'][i] / len(df)
    # skew / kurt
    for i in df.skew(axis = 0, skipna = True).index:
        try:
            summary[i]['skew'] = df.skew(axis = 0, skipna = True)[i]
        except:
            pass
    for i in df.kurtosis(axis = 0, skipna = True).index:
        try:
            summary[i]['kurt'] = df.kurtosis(axis = 0, skipna = True)[i]
        except:
            pass
    # quantitle
    for j in [0.01, 0.05, 0.10, 0.90, 0.95, 0.99]:
        for i in df.quantile(j, numeric_only=True).index:
            try:
                summary[i].loc[f"P{int(j*100)}"] = df.quantile(j, numeric_only=True)[i]
            except:
                pass
    # mean / max / min
    for j in ['max', 'min', 'mean']:
        for i in df.describe().loc[j].index:
            try:
                summary[i][j] = df.describe().loc[j][i]
            except:
                pass
    # Q1
    for i in df.describe().loc['25%'].index:
        try:
            summary[i]['Q1'] = df.describe().loc['25%'][i]
        except:
            pass
    # median
    for i in df.describe().loc['50%'].index:
        try:
            summary[i]['median'] = df.describe().loc['50%'][i]
        except:
            pass
    # Q3
    for i in df.describe().loc['75%'].index:
        try:
            summary[i]['Q3'] = df.describe().loc['75%'][i]
        except:
            pass
    # missing
    for i in pd.isnull(df).columns:
        try:
            summary[i]["missing"] = pd.isnull(df)[i].sum()
        except:
            pass
    # 0
    for i in df.columns:
        try:
            summary[i]["zero"] = (df[i] == 0).sum()
        except:
            pass
    # +
    for i in df.columns:
        try:
            summary[i]["positive"] = (df[i] > 0).sum()
        except:
            pass
    # -
    for i in df.columns:
        try:
            summary[i]["negative"] = (df[i] < 0).sum()
        except:
            pass
    # q-range
    summary.loc['q-range'] = summary.loc['Q3'] -summary.loc['Q1']
    return summary
# get the summarization of overall dataset
summary = get_summary(summary, df)
#summary.transpose().to_excel('summary.xlsx')
# get the summarization of each sector
sectors = ['Healthcare',
 'Basic Materials',
 'Consumer Defensive',
 'Industrials',
 'Financial Services',
 'Technology',
 'Consumer Cyclical',
 'Real Estate',
 'Energy',
 'Utilities',
 'Communication Services']
for i in sectors:
    df_group = df[df['Sector'] == i]
    # responsible for wikipedia and financial parameters part
    summary = pd.DataFrame(columns=['page_size_bytes', 'latest_edit_year_-_2018',
                                    'avg_day_between_edits', 'avg_edits_per_year', 'num_edits_2018',
                                    'num_edits_2017', 'edits_change_2017_2018', 'pageviews_60d',
                                    'num_sections', 'num_redir_links', 'num_references',
                                    'revenue_2018', 'revenue_2017', 'opincome_2018', 
                                    'opincome_2017', 'margin_2018', 'margin_2017', 'margin_change'], 
                           index=['coverage', 'min', 'P1', 'P5', 'P10', 'Q1', 'median', 
                                  'Q3', 'P90', 'P95', 'P99', 'max', 'mean', 'q-range', 
                                  'skew', 'kurt', 'zero', 'positive', 'negative', 'missing'])
    summary = summary.astype('float')
    summary = get_summary(summary, df_group)
#     summary.transpose().to_excel(f"{i}.xlsx")

#### Data Processing

Margin Change

In [ ]:
# convert margin change from numerical to binary, based on each sector
for i in sectors:
    try:
        df1 = pd.read_excel(f"sector_summary/{i}.xlsx", index_col="Unnamed: 0")
        for a in df[df['Sector'] == i].index:
            if df['margin_change'][a] >= df1['P90']['margin_change']:
                df['margin_change'][a] = 1
            else:
                df['margin_change'][a] = 0
    except:
        pass

Wikipedia Variables

In [ ]:
# calculate the event rate for wikipedia variables, based on each sector
wiki = df[['ticker', 'Name', 'Sector', 'page_size_bytes', 'latest_edit_year_-_2018',
          'avg_day_between_edits', 'avg_edits_per_year', 'num_edits_2018',
          'num_edits_2017', 'edits_change_2017_2018', 'pageviews_60d',
          'num_sections', 'num_redir_links', 'num_references', 'margin_change_90']]
cols = ['page_size_bytes', 'latest_edit_year_-_2018',
       'avg_day_between_edits', 'avg_edits_per_year', 'num_edits_2018',
       'num_edits_2017', 'edits_change_2017_2018', 'pageviews_60d',
       'num_sections', 'num_redir_links', 'num_references']
# because the coverage is less than 70%, convert them to categorical variables
for s in sectors:
    df1 = pd.read_excel(f'sector_summary/{s}.xlsx', index_col= 'Unnamed: 0')
    for a in cols:
        for b in wiki[wiki['Sector']==s].index:
            if wiki[a][b] > df1['P90'][a]:
                wiki[a][b] = "high"
            elif wiki[a][b] <= df1['P90'][a] and wiki[a][b] > df1['Q1'][a]:
                wiki[a][b] = "medium"
            elif wiki[a][b] <= df1['Q1'][a]:
                wiki[a][b] = "low"
            elif pd.isnull(wiki[a][b]):
                wiki[a][b] = "missing"

Event Rate

In [ ]:
# Calculate the event rate of margin change for all wikipedia variables, based on each sector
for a in cols:
    grouped = pd.DataFrame(wiki.groupby([a, 'Sector', 'margin_change_90']).size().unstack())
    grouped = grouped.replace(np.nan, 0)
    grouped['sum'] = grouped[0.0] + grouped[1.0]
    grouped[a] = grouped[1.0]/grouped['sum']
    grouped.to_excel(f"{a}.xlsx")

WOE Transformation

In [ ]:
# gather the event rate of each sector together and read it
event_rate = pd.read_excel('event_rate_90.xlsx')
# replace the categorical variables with WOE value for modeling process
for c in cols:
    for s in sectors:
        for row in wiki[wiki['Sector']==s].index:
            try:
                if wiki[c][row] == 'missing':
                    m = event_rate[(event_rate['Categories']=='missing')&(event_rate['Sector']==s)][c]
                    wiki[c][row] = math.log((1-m)/m)
                elif wiki[c][row] == 'high':
                    h = event_rate[(event_rate['Categories']=='high')&(event_rate['Sector']==s)][c]
                    wiki[c][row] = math.log((1-h)/h)
                elif wiki[c][row] == 'low':
                    l = event_rate[(event_rate['Categories']=='low')&(event_rate['Sector']==s)][c]
                    wiki[c][row] = math.log((1-l)/l)
                elif wiki[c][row] == 'medium':
                    d = event_rate[(event_rate['Categories']=='medium')&(event_rate['Sector']==s)][c]
                    wiki[c][row] = math.log((1-d)/d)
            except:
                continue
# replace the null and infinite value
for c in cols:
    for row in wiki.index:
        try:
            if math.isinf(wiki[c][row]) or math.isnan(wiki[c][row]):
                wiki[c][row] = 0
        except:
            continue
# wiki.to_excel('wiki_final_data.xlsx')